In [3]:
import mne
from mne import io, EvokedArray
from mne.datasets import sample
from mne.decoding import Vectorizer, get_coef

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score


import gc
import sys
import numpy as np
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

sys.path.append('/home/ginko/dreem')
from utils.load import load_data

In [4]:
raw = mne.io.read_raw("../data/mne/X_train_raw.fif", preload=True)
raw.filter(.5, 25, fir_design='firwin')

events = mne.find_events(raw, stim_channel='sleep_state', initial_event=True)

event_id = dict(awake=1, state_1=2, state_2=3, SWS=4, REM=5) 
tmin = 0
tmax = 6.
picks = mne.pick_types(raw.info, meg=False, eeg=True, ecg=False, misc=False,
                       stim=False, exclude='bads')
reject = dict(eeg=1.)

epochs = mne.Epochs(raw, events, event_id, tmin, tmax, proj=False,
                    picks=picks, baseline=None, reject=reject)

del raw
gc.collect();

Opening raw data file ../data/mne/X_train_raw.fif...
    Range : 0 ... 37031999 =      0.000 ... 148127.996 secs
Ready.
Reading 0 ... 37031999  =      0.000 ... 148127.996 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 25 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 25.00 Hz
- Upper transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 28.12 Hz)
- Filter length: 1651 samples (6.604 sec)

24688 events found
Event IDs: [1 2 3 4 5]
Not setting metadata
Not setting metadata
24688 matching events found
No baseline correction applied


In [5]:
X = epochs.get_data()
y = epochs.events[:, 2]

del epochs
gc.collect();

Loading data for 24688 events and 1501 original time points ...
1 bad epochs dropped


In [6]:
dim = X.shape
X = X.reshape([dim[0], dim[1] * dim[2]])
X = (X - X.mean()) / X.std()

In [10]:
kf = KFold(n_splits=4)
kf.get_n_splits(X)

scores = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    logreg = LogisticRegression(solver='lbfgs')
    logreg.fit(X_train, y_train)
    
    pred = logreg.predict(X_test)
    score = f1_score(y_test, pred, average='weighted')
    
    print(score)
    scores.append(score)
    


/home/ginko/.virtualenvs/ml/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.2690025335307313


/home/ginko/.virtualenvs/ml/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.21293938657407457


/home/ginko/.virtualenvs/ml/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.22947555596058006
0.2694427691382894


/home/ginko/.virtualenvs/ml/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
